<a href="https://colab.research.google.com/github/aithaprasad/NLP_Sentiment_Analysis/blob/master/Sentimental_Analysis_any_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [ ]:
data = pd.read_csv('train-v2.tsv', delimiter="\t", header = None, names = ['label', 'sentence'])

In [ ]:
data.head()

,label,sentence
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?


In [ ]:
data.isnull().values.any()

False

In [ ]:
data.shape

(78609, 2)

In [ ]:
X, y = data['sentence'], data['label']

In [ ]:
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X_dev, X_test, y_dev, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

X_train_tok = tokenizer.texts_to_sequences(X_train)
X_dev_tok = tokenizer.texts_to_sequences(X_dev)
X_test_tok = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_tok[60000]

[100, 60, 2777, 27, 2, 791, 2764, 5, 317]

In [ ]:
for item in X_train_tok:
  if 50000 in item:
    print("True")

In [ ]:
len(tokenizer.word_index)

61245

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train_pad = pad_sequences(X_train_tok, padding='post', maxlen=maxlen)
X_dev_pad = pad_sequences(X_dev_tok, padding='post', maxlen=maxlen)
X_test_pad = pad_sequences(X_test_tok, padding='post', maxlen=maxlen)

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, input_length=maxlen)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          6124600   
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 1)                 10001     
                                                                 
Total params: 6,134,601
Trainable params: 6,134,601
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train_pad, y_train, batch_size=128, epochs=6, verbose=1, validation_data=(X_dev_pad, y_dev))

Epoch 1/6
492/492 [==============================] - 42s 83ms/step - loss: 0.5712 - acc: 0.6960 - val_loss: 0.5162 - val_acc: 0.7425
Epoch 2/6
492/492 [==============================] - 40s 81ms/step - loss: 0.4147 - acc: 0.8152 - val_loss: 0.5205 - val_acc: 0.7456
Epoch 3/6
492/492 [==============================] - 41s 83ms/step - loss: 0.2635 - acc: 0.9008 - val_loss: 0.5725 - val_acc: 0.7367
Epoch 4/6
492/492 [==============================] - 40s 82ms/step - loss: 0.1473 - acc: 0.9552 - val_loss: 0.6465 - val_acc: 0.7262
Epoch 5/6
492/492 [==============================] - 40s 82ms/step - loss: 0.0870 - acc: 0.9768 - val_loss: 0.7258 - val_acc: 0.7175
Epoch 6/6
492/492 [==============================] - 41s 84ms/step - loss: 0.0558 - acc: 0.9866 - val_loss: 0.7972 - val_acc: 0.7159


In [ ]:
score = model.evaluate(X_test_pad, y_test, verbose=1)

246/246 [==============================] - 1s 2ms/step - loss: 0.7951 - acc: 0.7142


In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.795052707195282
Test Accuracy: 0.7141584753990173


In [ ]:
from keras.layers import Conv1D
cnn_model = Sequential()

#embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
cnn_model.add(embedding_layer)

cnn_model.add(Conv1D(128, 5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          6124600   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 6,188,857
Trainable params: 6,188,857
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cnn_history = cnn_model.fit(X_train_pad, y_train, batch_size=128, epochs=6, verbose=1, validation_data=(X_dev_pad, y_dev))

Epoch 1/6
492/492 [==============================] - 99s 199ms/step - loss: 0.3040 - acc: 0.8725 - val_loss: 0.6682 - val_acc: 0.7184
Epoch 2/6
492/492 [==============================] - 94s 191ms/step - loss: 0.1873 - acc: 0.9259 - val_loss: 0.8099 - val_acc: 0.7130
Epoch 3/6
492/492 [==============================] - 94s 191ms/step - loss: 0.1172 - acc: 0.9570 - val_loss: 0.9912 - val_acc: 0.6981
Epoch 4/6
492/492 [==============================] - 95s 193ms/step - loss: 0.0756 - acc: 0.9731 - val_loss: 1.2105 - val_acc: 0.6979
Epoch 5/6
492/492 [==============================] - 94s 191ms/step - loss: 0.0475 - acc: 0.9846 - val_loss: 1.4255 - val_acc: 0.6934
Epoch 6/6
492/492 [==============================] - 94s 190ms/step - loss: 0.0295 - acc: 0.9912 - val_loss: 1.6336 - val_acc: 0.6927


In [ ]:
score = cnn_model.evaluate(X_test_pad, y_test, verbose=1)

246/246 [==============================] - 4s 16ms/step - loss: 1.5763 - acc: 0.6913


In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 1.5762993097305298
Test Accuracy: 0.6912606358528137
